## *For up-to-date plotting script use plots/combineFiles.ipynb*

In [ ]:
import pandas as pd
import numpy as np
from coffea import util
import itertools
import os, sys
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use("CMS")

sys.path.append('../python/')
from functions import loadCoffeaFile, getLabelMap, getCoffeaFilenames, plotBackgroundEstimate, getHist


## analysis categories

In [ ]:
IOVs = ['2016']

# analysis categories #

label_dict = loadCoffeaFile()['analysisCategories']

label_to_int_dict = {label: i for i, label in label_dict.items()}

signal_cats = [ i for label, i in label_to_int_dict.items() if '2t' in label]
pretag_cats = [ i for label, i in label_to_int_dict.items() if 'pre' in label]
antitag_cats = [ i for label, i in label_to_int_dict.items() if 'at' in label]




In [ ]:
label_dict

## scale factors and IOV

In [ ]:
IOVs = ['2016']

lumi = {
    "2016APV": 19800.,
    "2016": 16120., #35920 - 19800
    "2016all": 35920,
    "2017": 41530.,
    "2018": 59740.
}

t_BR = 0.6741
ttbar_BR = 0.4544 #PDG 2019
ttbar_xs1 = 831.76 * (0.09210) #pb For ttbar mass from 700 to 1000
ttbar_xs2 = 831.76 * (0.02474) #pb For ttbar mass from 1000 to Inf
toptag_sf = 0.9
toptag_kf = 1.0 #0.7
qcd_xs = 13700000.0 #pb From https://cms-gen-dev.cern.ch/xsdb



## save plots

In [ ]:
directories = [
    'images/png/closureTestQCD/2016all',
    'images/png/closureTestQCD/2016APV',
    'images/png/closureTestQCD/2016',
    'images/png/closureTestQCD/2017',
    'images/png/closureTestQCD/2018',
    'images/pdf/closureTestQCD/2016all',
    'images/pdf/closureTestQCD/2016APV',
    'images/pdf/closureTestQCD/2016',
    'images/pdf/closureTestQCD/2017',
    'images/pdf/closureTestQCD/2018',
]


for path in directories:
    if not os.path.exists(path):
        os.makedirs(path)

## coffea files

In [ ]:
tagger='DeepAK8 top tagger'
# tagger='CMS top tagger'


coffeaFiles = getCoffeaFilenames()

# coffeaFiles



# Background Estimate 
### with msd cut

In [ ]:
IOV = '2016'

fig, (ax1, ax2) = plt.subplots(nrows=2, height_ratios=[3, 1])

sig = util.load(coffeaFiles['QCD']['unweighted'][IOV])
bkg = util.load(coffeaFiles['QCD']['weighted'][IOV])

qcdsf = {
    'sig': lumi[IOV] * qcd_xs / sig['cutflow']['sumw'],
    'bkg': lumi[IOV] * qcd_xs / bkg['cutflow']['sumw'],
}

hsig = sig['ttbarmass'][{'anacat':signal_cats}][{'anacat':sum, 'systematic':sum}] * qcdsf['sig']
hbkg = bkg['ttbarmass'][{'anacat':pretag_cats}][{'anacat':sum, 'systematic':sum}] * qcdsf['bkg']


hep.cms.label('', data=True, lumi='{0:0.1f}'.format(lumi[IOV]/1000), year=IOV, loc=2, fontsize=20, ax=ax1)
hep.cms.text('Preliminary'+'\n'+r'$\Delta y$ inclusive'+'\n'+r'btag inclusive''\n'+r'DeepAK8 tagger', loc=2, fontsize=20, ax=ax1)

hep.histplot(hsig, histtype='errorbar', color='black', label='QCD SR', ax=ax1)
hep.histplot(hbkg, histtype='fill', color='xkcd:pale gold', label='QCD Bkg Est', ax=ax1)


ratio_plot =  hsig / hbkg.values()
hep.histplot(ratio_plot, ax=ax2, histtype='errorbar', color='black')
ax2.set_ylim(0,2)
ax2.axhline(1, color='black', ls='--')
ax2.set_ylabel('Data/Bkg')


ax1.legend()
ax1.set_ylabel('Events')
ax1.set_yscale('log')
ax1.set_ylim(1e-1, 1e13)
ax1.set_xlim(900,6000)
ax1.set_xlabel('')


# Transfer Function

In [ ]:
# fit N events from CR to N events from SR


# loose to fail | pretag to antitag

def rlf(x, mt, p0, p1):
    
    return x*(p0 + p1 * mt)


# pass to loose | signal to pretag

def rtl(x, mt, mtt, p2, p3, p4, p5):
    
    return x * (p2 + p3 * mt + p4 * mt * mt) * (p5 * mtt) 
    

In [ ]:
qcd = loadCoffeaFile()


mtt_vs_mt = getHist('mtt_vs_mt', 'JetHT', False, '2016', sum_axes=[], integrate_axes={})



h2dsr = mtt_vs_mt[{'anacat':signal_cats}][{'anacat':sum}]
h2dcr = mtt_vs_mt[{'anacat':antitag_cats}][{'anacat':sum}]
h2dpr = mtt_vs_mt[{'anacat':pretag_cats}][{'anacat':sum}]





qcdsf = lumi[IOV] * qcd_xs / qcd['cutflow']['sumw']

hsig_mtt = h2dsr[{'jetmass':sum}]# * qcdsf
hbkg_mtt = h2dcr[{'jetmass':sum}]# * qcdsf


Nsr = h2dsr.values()#*qcdsf
Ncr = h2dcr.values()#*qcdsf
Npr = h2dpr.values()#*qcdsf





## signal region $N_{events}$  / pretag region $N_{events}$

for transfer function $n_{tight} = n_{loose} * R$

In [ ]:
(h2dsr/h2dpr.values()).plot2d()

## pretag region $N_{events}$  / antitag region $N_{events}$

for transfer function $n_{loose} = n_{fail} * R$

In [ ]:
(h2dpr/h2dcr.values()).plot2d()

In [ ]:

bins_mt = h2dsr.axes['jetmass'].edges[:-1]
bins_mtt = h2dsr.axes['ttbarmass'].edges[:-1]


mtt, mt = np.meshgrid(bins_mtt, bins_mt)



# flatten arrays for fitting
f_mt  =  mt.flatten()
f_mtt = mtt.flatten()
f_nsr = Nsr.flatten()
f_ncr = Ncr.flatten()
f_npr = Npr.flatten()


In [ ]:
# loose to fail | pretag to antitag


y = np.where(f_ncr > 0, f_npr/f_ncr, 0)

popt, pcov = curve_fit(lambda x, p0, p1: rlf(x, f_mt, p0, p1), 
                       f_ncr, 
                       f_npr, 
                       p0=[1,1])


p0, p1 = popt

f_rlf = rlf(np.ones_like(Ncr), mt, p0, p1)


In [ ]:

# pass to loose | signal to pretag


y = np.where(f_npr > 0, f_nsr/f_npr, 0)





popt, pcov = curve_fit(lambda x, p2, p3, p4, p5: rtl(x, f_mt, f_mtt, p2, p3, p4, p5), 
                       f_npr,
                       f_nsr,
                       p0=[1,1,1,1], bounds=[-10,10])


p2, p3, p4, p5 = popt

f_rtl = rtl(np.ones_like(Ncr), mt, mtt, p2, p3, p4, p5)

## transfer function plots

In [ ]:
fig = plt.figure(figsize=(20, 14))
ax1 = fig.add_subplot(projection='3d')

x = mtt.flatten() #f_mtt
y = mt.flatten() #f_mt
z = f_rlf.flatten()

bottom = np.zeros_like(z) #mtt_vs_mt.values()[1]

dx = 200
dy = 20

cmap = plt.cm.get_cmap('jet')
hmax = np.max(z)
hmin = np.min(z)

# hmin = 0.0

rgba = [cmap((k-hmin)/hmax) for k in z] 

ax1.bar3d(x, y, bottom, dx, dy, z, color=rgba)
ax1.set_xlim(8000,0)


ax1.zaxis.set_rotate_label(False)
ax1.set_xlabel(r'$m_{tt}$', labelpad=20, loc='right')
ax1.set_ylabel(r'$m_t$', labelpad=20)
ax1.set_zlabel(r'$R_{L/F}$', labelpad=30)#, loc='top')#, rotation=180)

ax1.dist = 12


In [ ]:
fig = plt.figure(figsize=(15, 14))
ax1 = fig.add_subplot(projection='3d')

x = mtt.flatten()#f_mtt
y = mt.flatten()#f_mt

z = f_rtl.flatten()
z = np.where(z < 0, 0, z)

bottom = np.zeros_like(z) #mtt_vs_mt.values()[1]

dx = 200
dy = 20

cmap = plt.cm.get_cmap('jet')
hmax = np.max(z)
hmin = np.min(z)

# hmin = 0.0
# hmax = 0.05
rgba = [cmap((k-hmin)/hmax) for k in z] 

ax1.bar3d(x, y, bottom, dx, dy, z, color=rgba)#, zorder=0)
ax1.set_xlim(8000,0)
# ax1.set_zlim(0.01,0.12)

ax1.zaxis.set_rotate_label(False)
ax1.set_xlabel(r'$m_{tt}$', labelpad=20, loc='right')
ax1.set_ylabel(r'$m_t$', labelpad=20)
ax1.set_zlabel(r'$R_{T/L}$', labelpad=30)#, loc='top')#, rotation=180)

ax1.dist = 13

plt.show()


In [ ]:
print('p0 {0:0.2e}'.format(p0))
print('p1 {0:0.2e}'.format(p1))
print('p2 {0:0.2e}'.format(p2))
print('p3 {0:0.2e}'.format(p3))
print('p4 {0:0.2e}'.format(p4))
print('p5 {0:0.2e}'.format(p5))


# Regions multiplied by transfer function

In [ ]:
antitag2d  = getHist('mtt_vs_mt', 'JetHT', False, IOV, sum_axes=['anacat'], integrate_axes={'anacat':antitag_cats})
pretag2d  = getHist('mtt_vs_mt', 'JetHT', False, IOV, sum_axes=['anacat'], integrate_axes={'anacat':pretag_cats})
signal2d   = getHist('mtt_vs_mt', 'JetHT', False, IOV, sum_axes=['anacat'], integrate_axes={'anacat':signal_cats})


fig, (ax1, ax2) = plt.subplots(nrows=2, height_ratios=[3, 1])

antitag2d[{'jetmass':sum}].plot1d(ax=ax1, label='antitag')
(antitag2d * f_rlf)[{'jetmass':sum}].plot1d(ax=ax1, label='antitag scaled')
antitag_ratio = antitag2d[{'jetmass':sum}]/(antitag2d * f_rlf)[{'jetmass':sum}].values()
antitag_ratio.plot1d(ax=ax2)


ax1.legend()
ax1.set_xlabel('')
ax1.set_ylabel('Events')
ax2.set_ylabel('Data / Bkg Est')



In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, height_ratios=[3, 1])

pretag2d[{'jetmass':sum}].plot1d(ax=ax1, label='pretag')
(pretag2d * f_rlf)[{'jetmass':sum}].plot1d(ax=ax1, label='pretag scaled')
pretag_ratio = pretag2d[{'jetmass':sum}]/(pretag2d * f_rlf)[{'jetmass':sum}].values()
pretag_ratio.plot1d(ax=ax2)


ax1.legend()
ax1.set_xlabel('')
ax1.set_ylabel('Events')
ax2.set_ylabel('Data / Bkg Est')



In [ ]:


fig, (ax1, ax2) = plt.subplots(nrows=2, height_ratios=[3, 1])

signal2d[{'jetmass':sum}].plot1d(ax=ax1, label='signal')
(signal2d * f_rlf)[{'jetmass':sum}].plot1d(ax=ax1, label='signal scaled')
signal_ratio = signal2d[{'jetmass':sum}]/(signal2d * f_rlf)[{'jetmass':sum}].values()
signal_ratio.plot1d(ax=ax2)


ax1.legend()
ax1.set_xlabel('')
ax1.set_ylabel('Events')
ax2.set_ylabel('Data / Bkg Est')



In [ ]:
f_rlf

# Transfer Function + Mistag Rate

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, height_ratios=[3, 1])


bins = hbkg_mtt.axes['ttbarmass'].edges[:-1]


IOV = '2016'

httbar = getHist('ttbarmass', 'TTbar', False, IOV, sum_axes=['anacat','systematic'], integrate_axes={'anacat':signal_cats})
hdata  = getHist('ttbarmass', 'JetHT', False, IOV, sum_axes=['anacat','systematic'], integrate_axes={'anacat':signal_cats})


h_mtt_mt = getHist('mtt_vs_mt', 'JetHT', False, IOV, sum_axes=['anacat'], integrate_axes={'anacat':pretag_cats})
hntmj_2d = h_mtt_mt * f_rlf
hntmj = hntmj_2d[{'jetmass':sum}]


hbkg = httbar + hntmj# * np.abs(np.average(f_rtl, axis=0)) #* np.abs(np.average(f_rlf, axis=1))


hep.histplot(hdata, histtype='errorbar', color='k', ax=ax1, label='Data')
hep.histplot(hbkg, histtype='fill', color='xkcd:pale gold', label='NTMJ Bkg Est', ax=ax1)
hep.histplot(httbar, histtype='fill', color='xkcd:deep red', label='TTbar', ax=ax1)


ax1.set_yscale('log')


ax1.set_ylabel('Events')

ratio_plot = hdata / h2.values()


hep.histplot(ratio_plot, ax=ax2, histtype='errorbar', color='black')
ax2.set_ylim(0,2)
ax2.axhline(1, color='black', ls='--')
ax2.set_ylabel('Data/Bkg')
ax2.set_xlabel(ax1.get_xlabel())
ax1.set_xlabel('')


ax1.legend(loc=1)




## transfer function by categories

In [ ]:
IOV = '2016'


cats = ['0bcen', '0bfwd', '1bcen', '1bfwd', '2bcen', '2bfwd']

for cat in cats:
    
    
    fig, (ax1, ax2) = plt.subplots(nrows=2, height_ratios=[3, 1])


    signal_cat = label_to_int_dict['2t'+cat]
    pretag_cat = label_to_int_dict['pret'+cat]
    
    
    httbar = getHist('ttbarmass', 'TTbar', False, IOV, sum_axes=['systematic'], integrate_axes={'anacat':signal_cat})
    hdata  = getHist('ttbarmass', 'JetHT', False, IOV, sum_axes=['systematic'], integrate_axes={'anacat':signal_cat})


    h_mtt_mt = getHist('mtt_vs_mt', 'JetHT', False, IOV, sum_axes=[], integrate_axes={'anacat':pretag_cat})
    hntmj_2d = h_mtt_mt * f_rlf
    hntmj = hntmj_2d[{'jetmass':sum}]


    hbkg = httbar + hntmj# * np.abs(np.average(f_rtl, axis=0)) #* np.abs(np.average(f_rlf, axis=1))





    hep.histplot(hdata, histtype='errorbar', color='k', ax=ax1, label='Data')
    hep.histplot(hbkg, histtype='fill', color='xkcd:pale gold', label='NTMJ Bkg Est', ax=ax1)
    hep.histplot(httbar, histtype='fill', color='xkcd:deep red', label='TTbar', ax=ax1)


    ax1.set_yscale('log')


    ax1.set_ylabel('Events')

    ratio_plot = hdata / h2.values()


    hep.histplot(ratio_plot, ax=ax2, histtype='errorbar', color='black')
    ax2.set_ylim(0,2)
    ax2.axhline(1, color='black', ls='--')
    ax2.set_ylabel('Data/Bkg')
    ax2.set_xlabel(ax1.get_xlabel())
    ax1.set_xlabel('')


    ax1.legend(loc=1)
    
    
    plt.plot()


    
    



In [ ]:
np.save('save',f_rlf)

In [ ]:
f_rlf.shape